## Hot, nasty speed

We will talk about some simple ways to speed up your computations in python.

In [ ]:
import sys
sys.version

We can vectorize basic functions, just like we would with ordinary equations.

In [15]:
import os
os.environ

environ({'WINDOWSSDKVERSION': '\\', 'LIBPATH': 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\BuildTools\\VC\\Tools\\MSVC\\14.15.26726\\lib\\x64;C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\BuildTools\\VC\\Tools\\MSVC\\14.15.26726\\lib\\x86\\store\\references;C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319;C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\BuildTools\\VC\\Tools\\MSVC\\14.15.26726\\lib\\x64;C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\BuildTools\\VC\\Tools\\MSVC\\14.15.26726\\lib\\x86\\store\\references;C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319;', 'PROCESSOR_IDENTIFIER': 'Intel64 Family 6 Model 142 Stepping 9, GenuineIntel', 'VCTOOLSREDISTDIR': 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\BuildTools\\VC\\Redist\\MSVC\\14.15.26706\\', '__DOTNET_ADD_64BIT': '1', 'COMPUTERNAME': 'LAPTOP-KPMLGEVS', 'WINDOWSSDK_EXECUTABLEPATH_X64': 'C:\\Program Files (x86)\\Microsoft SDKs\\Windows\\v10.0A\\bin\\NETFX 4.6.1 Tools\

In [1]:
%matplotlib inline
import numpy as np
from warnings import warn

In [ ]:
y = np.array([1.,2,3,4])
x = np.array([5.,6,7,8])

Numpy arrays work well with **simple** functions.

In [ ]:
def SSE(x,y):
    return (x-y)**2

In [ ]:
SSE(x,y)

However, lets look at a more complicated function. Absolute Value.

In [ ]:
def absDistance(x,y):
    if x < y:
        return y-x
    else:
        return x-y

In [ ]:
try:
    absDistance(y,x)
except:
    raise ValueError('Nah Dude.')

In [ ]:
dist = np.zeros(4)
for i in range(len(x)):
    dist[i] = absDistance(y[i], x[i])
dist

In [ ]:
v_absDistance = np.vectorize(absDistance)

In [ ]:
v_absDistance(y,x)

In [ ]:
x = np.arange(0,1e7)
y = np.arange(0,1e7) + 4

In [ ]:
%%time
dist = np.zeros(len(x))
for i in range(len(x)):
    dist[i] = absDistance(y[i], x[i])
dist

In [ ]:
%%time
v_absDistance(y,x)

Python is an interpreted language, meaning the code is compiled line by line or as needed. C++ and Fortran are compiled codes, meaning their compilers are closer to machine language. Therefore, they are faster. Luckily, there are libraries in Python that perform just-in-time compilations to bridge the gap in speed.

In [2]:
from numba import jit, vectorize, float64
import cython
import multiprocessing as mp

In [ ]:
def factorial(n):
    if(n==1):
        return 1
    else:
        return n*factorial(n-1)

In [ ]:
%%time
factorial(1000)

In [ ]:
factorial(3000)

In [ ]:
%%time
n = 200000
fact = 1
while(n!=1):
    fact = fact*n
    n-=1

Compile in C.

In [3]:
%load_ext Cython

In [16]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print(a)

DistutilsPlatformError: Unable to find vcvarsall.bat

In [ ]:
%%writefile fact.pyx

cdef int n = 200000
cdef int fact = 1
while(n!=1):
    fact = fact*n
    n -= 1

In [11]:
%%writefile setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("fact.pyx")
)

Overwriting setup.py


In [15]:
%run -i setup.py build_ext --inplace

running build_ext
building 'fact' extension


SystemExit: error: Unable to find vcvarsall.bat

In [ ]:
%run -i fact.pyx

In [ ]:
from numba import jit

@jit
def numba_fact(n):
    fac=1
    while(n!=1):
        fac = fac*n
        n-=1
    return fac

In [ ]:
%%time
fac = numba_fact(200000)

In [ ]:
from numba import int32

@vectorize([float64(float64, float64), int32(int32,int32)])
def numba_ad(x,y):
    if x < y:
        return y-x
    else:
        return x-y

In [ ]:
%%time
numba_ad(x,y)

In [ ]:
%%time
ncpus = mp.cpu_count()
p = mp.Pool(ncpus-1)
result = p.map(fact, [200000])

In [ ]:
%%time
from __future__ import print_function, division, absolute_import
from matplotlib.pylab import imshow, jet, show, ion


@jit
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            return i

    return 255

@jit
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x] = color

    return image

image = np.zeros((500 * 2, 750 * 2), dtype=np.uint8)
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 40)
imshow(image)
#jet()
#ion()
show()